## Teste 1
##### Programação grosseira em busca dos melhores dados e dos melhores modelos

In [293]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from warnings import simplefilter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 8)
#sns.set_style('whitegrid')
sns.set_style('white')
sns.set_context('poster')

In [294]:
df = pd.read_csv('datasets/dataset_cler.csv', sep=";", encoding='utf-16')
del df['Ref']
df.shape

(1445, 555)

In [295]:
df.head(2)

,ampl_29,desv_amp_29,body_29,body_per_29,pavio_sup_29,pavio_sup_per_29,pavio_inf_29,pavio_inf_per_29,high_dist_29,low_dist_29,...,pavio_inf_h2_1,pavio_inf_per_h2_1,high_dist_h2_1,low_dist_h2_1,type_h2_1,atr,qt_bars,oper,take,status
0,15.0,0.43,10.0,66.67,5.0,33.33,0.0,0.00,10.0,30.0,...,215.0,47.78,615.0,620.0,up,58.21,5,Sell,-40.0,Loss
1,90.0,1.29,5.0,5.56,15.0,16.67,70.0,77.78,45.0,65.0,...,215.0,47.78,615.0,620.0,up,59.64,5,Sell,-40.0,Loss


In [296]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [297]:
df.drop(df.loc[(df['status']!='Loss') & (df['status']!='Gain')].index.values, inplace=True)

In [298]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [299]:
#train_features = df.columns.difference(['take', 'status', 'oper'])
X = df.drop(['take','status','oper'], axis=1) #df[train_features]
y = df['status']

x_columns = X.columns

In [300]:
index_features_category = []
for index, col in enumerate(X.columns):
    if X.dtypes[index]=='object':
        index_features_category.append(index)
index_features_category[:10]

[10, 21, 32, 43, 54, 65, 76, 87, 98, 109]

In [301]:
encoder = LabelEncoder()
encoder.fit(['inside', 'up', 'down'])
columns_categorical = X.select_dtypes(include=['object']).columns
for col_cat in columns_categorical:
    X[col_cat] = encoder.transform(X[col_cat])

ohe = OneHotEncoder(categorical_features=index_features_category)
X = ohe.fit_transform(X).toarray()

In [302]:
def fit(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return y_pred 

def fitAll(X_train, X_test, y_train, y_test, algorithms):
    similaridade = []
    for item in algorithms:
        name = item[0]
        algorithm = item[1]
        
        algorithm.fit(X_train, y_train)
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #print(name, ' => ', accuracy)
        similaridade.append((accuracy, name))
        
    similaridade.sort()
    similaridade.reverse()
    
    for j in similaridade:
        print(j)

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [303]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

standar = StandardScaler()
standar.fit(X)
X = standar.fit_transform(X)

In [304]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [305]:
algorithms = []

algorithms.append(['DecisionTreeClassifier', DecisionTreeClassifier(random_state=0)])
algorithms.append(['RandomForestClassifier', RandomForestClassifier(n_estimators=10, random_state=0)])
algorithms.append(['GaussianNB', GaussianNB()])
algorithms.append(['RidgeClassifier', RidgeClassifier()])
algorithms.append(['MLPClassifier', MLPClassifier(random_state=True, max_iter=1000, learning_rate_init=0.0001, hidden_layer_sizes=(89))])
algorithms.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
algorithms.append(['SVC', SVC(gamma='auto')])
algorithms.append(['KNeighborsClassifier', KNeighborsClassifier()])
algorithms.append(['LogisticRegression', LogisticRegression()])

fitAll(X_train, X_test, y_train, y_test, algorithms)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(0.5391705069124424, 'KNeighborsClassifier')
(0.5069124423963134, 'GradientBoostingClassifier')
(0.4792626728110599, 'SVC')
(0.46774193548387094, 'DecisionTreeClassifier')
(0.46543778801843316, 'RandomForestClassifier')
(0.46543778801843316, 'LogisticRegression')
(0.4631336405529954, 'RidgeClassifier')
(0.45852534562211983, 'MLPClassifier')
(0.4377880184331797, 'GaussianNB')


test = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000)
y_pred = fit(X_train, X_test, y_train, y_test, test)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

In [306]:
y_test[300:301]

462    Loss
Name: status, dtype: object

In [307]:
X_test[300]

array([0.15436242, 0.33628319, 0.1097561 , 0.36      , 0.42424242,
       0.60305837, 0.03636364, 0.09066183, 0.25      , 0.0625    ,
       1.        , 0.01941748, 0.00340832, 0.00502513, 0.1667    ,
       0.1       , 0.35552   , 0.08571429, 0.53333333, 0.06622517,
       0.04639175, 1.        , 0.12962963, 0.31578947, 0.05681818,
       0.3125    , 0.07142857, 0.13541328, 0.32142857, 0.64285714,
       0.03883495, 0.08108108, 1.        , 0.14285714, 0.08447205,
       0.20634921, 0.9286    , 0.02777778, 0.07820372, 0.        ,
       0.        , 0.11340206, 0.12621359, 1.        , 0.11504425,
       0.090625  , 0.12698413, 0.5333    , 0.15      , 0.44999438,
       0.03846154, 0.07336927, 0.07017544, 0.046875  , 0.        ,
       0.06593407, 0.03643725, 0.05952381, 0.625     , 0.08333333,
       0.29543843, 0.04      , 0.13393335, 0.17647059, 0.06666667,
       0.        , 0.11340206, 0.1678487 , 0.10588235, 0.6923    ,
       0.13793103, 0.32969035, 0.        , 0.        , 0.02222